In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from datetime import datetime

In [2]:
def print_full(x):
    pd.set_option('display.max_seq_items', 10000)
    pd.set_option('display.max_rows', 10000)
    pd.set_option('display.max_columns', 10000)
    print(x)
    pd.reset_option('display.max_seq_items')
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')

In [3]:
train=pd.read_csv("C:/Users/10644292/AppData/Local/Programs/Python/Python36/Scripts/Data Science Assignments/AMEX/train_amex/train.csv", sep = ',')
test=pd.read_csv("C:/Users/10644292/AppData/Local/Programs/Python/Python36/Scripts/Data Science Assignments/AMEX/test_LNMuIYp/test.csv", sep = ',')
hist=pd.read_csv("C:/Users/10644292/AppData/Local/Programs/Python/Python36/Scripts/Data Science Assignments/AMEX/train_amex/historical_user_logs.csv", sep = ',')

In [4]:
hist.loc[hist['action']=='interest','action']=1
hist.loc[hist['action']=='view','action']=0
hist_user_prod=hist.groupby(['user_id','product'])['action'].agg(['mean']).reset_index()
hist_user=hist.groupby(['user_id'])['action'].agg(['mean']).reset_index()
hist_user.rename(columns={'mean': 'user_pref'}, inplace=True)
hist_user_prod.rename(columns={'mean': 'user_prod_pref'}, inplace=True)

In [5]:
test['is_click']=0
test['train']=0
train['train']=1
test1 = test.append(train, ignore_index=True)

In [6]:
test1.drop(['product_category_2'], axis = 1, inplace = True)
test1.fillna(value={'user_group_id':2,'gender': "Male", 'age_level':2, 'user_depth':1,'city_development_index':1}, inplace=True)

In [7]:
test1['RN'] = test1.sort_values(['user_id','DateTime'], ascending=[True,True]).groupby(['user_id']).cumcount()

In [8]:
test1['Rep_usr']=0
test1.loc[test1['user_id'].isin(test1.loc[test1['RN']==2,'user_id']),'Rep_usr']=1

In [9]:
a=test1.copy()
a['Cum_Click'] = a.sort_values(['user_id','DateTime'], ascending=[True,True]).groupby(['user_id']).agg({'is_click':'cumsum'}) 
a['Cum_Click2']=a.sort_values(['user_id','DateTime'], ascending=[True,True]).groupby('user_id')['Cum_Click'].shift(1)
a.loc[a['Cum_Click2']!=a['Cum_Click2'],'Cum_Click2']=0
test1['Cum_Click']=a['Cum_Click2']

In [10]:
test1['DateTime']=test1['DateTime'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M'))
test1['Day'] = test1['DateTime'].map(lambda x: x.day)
test1['hour'] = test1['DateTime'].map(lambda x: x.hour)

In [11]:
test1['RN_Camp'] = test1.sort_values(['user_id','campaign_id','DateTime'], ascending=[True,True,True]).groupby(['user_id','campaign_id']).cumcount()

In [12]:
test1['RN_P'] = test1.sort_values(['user_id','product','DateTime'], ascending=[True,True,True]).groupby(['user_id','product']).cumcount()

In [13]:
test1['RN_Web'] = test1.sort_values(['user_id','webpage_id','DateTime'], ascending=[True,True,True]).groupby(['user_id','webpage_id']).cumcount()

In [14]:
test1['RN_day'] = test1.sort_values(['user_id','Day','DateTime'], ascending=[True,True,True]).groupby(['user_id','Day']).cumcount()

In [15]:
test1['RN_hour'] = test1.sort_values(['user_id','hour','DateTime'], ascending=[True,True,True]).groupby(['user_id','hour']).cumcount()

In [16]:
test1['RN_Day_hour'] = test1.sort_values(['user_id','Day','hour','DateTime'], ascending=[True,True,True,True]).groupby(['user_id','Day','hour']).cumcount()

In [17]:
a=test1.copy()
a['Day_Cum_Click'] = a.sort_values(['user_id','Day','DateTime'], ascending=[True,True,True]).groupby(['user_id','Day']) .agg({'is_click':'cumsum'}) 
a['Day_Cum_Click2']=a.sort_values(['user_id','Day','DateTime'], ascending=[True,True,True]).groupby(['user_id','Day'])['Day_Cum_Click'].shift(1)
a.loc[a['Day_Cum_Click2']!=a['Day_Cum_Click2'],'Day_Cum_Click2']=0
test1['Day_Cum_Click']=a['Day_Cum_Click2']

In [18]:
a=test1
a['t_lag'] = a.sort_values(['user_id','DateTime'], ascending=[True,True]).groupby('user_id')['DateTime'].shift(1)
a.loc[a['t_lag']!=a['t_lag'],'t_lag']=a['DateTime']
a['t_diff']=(a['DateTime']-a['t_lag']).astype('timedelta64[h]')
test1['t_diff']=a['t_diff']

In [19]:
test1=pd.merge(test1, hist_user, how = 'left', left_on = 'user_id', right_on = 'user_id')
test1=pd.merge(test1, hist_user_prod, how = 'left', left_on = ['user_id','product'], right_on = ['user_id','product'])
test1.loc[test1['user_prod_pref']!=test1['user_prod_pref'],'user_prod_pref']=0
test1.loc[test1['user_pref']!=test1['user_pref'],'user_pref']=0

In [20]:
test1.drop(['t_lag'], axis = 1, inplace = True)
test1.drop(['DateTime'], axis = 1, inplace = True)
test1.drop(['user_id'], axis = 1, inplace = True)

In [21]:
df=test1.copy()
df['campaign_id'] = df['campaign_id'].astype('str') 
df['webpage_id'] = df['webpage_id'].astype('str')
df['product_category_1'] = df['product_category_1'].astype('str')
df['user_group_id'] = df['user_group_id'].astype('str')
df['age_level'] = df['age_level'].astype('str')
df['user_depth'] = df['user_depth'].astype('str')
df['city_development_index'] = df['city_development_index'].astype('str')
df['hour'] = df['hour'].astype('str')

In [22]:
df1=pd.get_dummies(df)

In [23]:
testing_dt=df1.loc[df1['train']==0,].reset_index()
df1=df1.loc[df1['train']==1,]
validation_dt=df1.loc[df1['Day']>=6,]
df1=df1.loc[df1['Day']<6,]

#df1.drop(['Day'], axis = 1, inplace = True)
#validation_dt.drop(['Day'], axis = 1, inplace = True)
#testing_dt.drop(['Day'], axis = 1, inplace = True)
testing_dt.drop(['train'], axis = 1, inplace = True)
df1.drop(['train'], axis = 1, inplace = True)
validation_dt.drop(['train'], axis = 1, inplace = True)

In [33]:
y=df1["is_click"]
X=df1[['var_1', 'RN', 'Rep_usr', 'Day',
       'Cum_Click', 'user_prod_pref','user_pref','RN_Camp', 
       'RN_P', 'RN_Web', 'RN_day', 'RN_hour', 
       'RN_Day_hour','Day_Cum_Click', 't_diff',
       'product_A', 'product_B', 'product_C', 'product_D', 'product_E',
       'product_F', 'product_G', 'product_H', 'product_I', 'product_J',
       'campaign_id_105960', 'campaign_id_118601', 'campaign_id_359520',
       'campaign_id_360936', 'campaign_id_396664', 'campaign_id_404347',
       'campaign_id_405490', 'campaign_id_414149', 'campaign_id_82320',
       'campaign_id_98970', 'webpage_id_11085', 'webpage_id_13787',
       'webpage_id_1734', 'webpage_id_28529', 'webpage_id_45962',
       'webpage_id_51181', 'webpage_id_53587', 'webpage_id_60305',
       'webpage_id_6970', 'product_category_1_1', 'product_category_1_2',
       'product_category_1_3', 'product_category_1_4', 'product_category_1_5',
       'user_group_id_0.0', 
       #'user_group_id_0.1', 
       'user_group_id_1.0',
       'user_group_id_10.0', 'user_group_id_11.0', 'user_group_id_12.0',
       'user_group_id_2.0', 'user_group_id_3.0', 'user_group_id_4.0',
       'user_group_id_5.0', 'user_group_id_6.0', 'user_group_id_7.0',
       'user_group_id_8.0', 'user_group_id_9.0', 'gender_Female',
       'gender_Male', 
       #'gender_N',
       'age_level_0.0', 
       #'age_level_0.1',
       'age_level_1.0', 'age_level_2.0', 'age_level_3.0', 'age_level_4.0',
       'age_level_5.0', 'age_level_6.0', 
       #'user_depth_0.0', 
       'user_depth_1.0',
       'user_depth_2.0', 'user_depth_3.0', 
       #'city_development_index_0.0',
       'city_development_index_1.0', 'city_development_index_2.0',
       'city_development_index_3.0', 'city_development_index_4.0', 'hour_0',
       'hour_1', 'hour_10', 'hour_11', 'hour_12', 'hour_13', 'hour_14',
       'hour_15', 'hour_16', 'hour_17', 'hour_18', 'hour_19', 'hour_2',
       'hour_20', 'hour_21', 'hour_22', 'hour_23', 'hour_3', 'hour_4',
       'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9']]

In [ ]:
klghbjkl;,kijhy

In [37]:
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
Final_model=RandomForestClassifier(bootstrap=True, criterion='gini', n_jobs=-1,random_state=123, verbose=1, min_samples_leaf=700)#max_depth=9)
Final_model.fit(X, y)


print("Performance metrics for Training Data")
concordance_matrix=pd.crosstab(y, Final_model.predict(X))
#print("Accuracy = ",((concordance_matrix[0][0]+concordance_matrix[1][1])/(concordance_matrix[0][0]+concordance_matrix[1][1]+concordance_matrix[1][0]+concordance_matrix[0][1])))
#print("Precision = ", (concordance_matrix[1][1]/(concordance_matrix[1][1]+concordance_matrix[1][0])))
#print("Recall = ", (concordance_matrix[1][1]/(concordance_matrix[1][1]+concordance_matrix[0][1])))
print('\n',"Concordance Matrix",'\n',concordance_matrix)
print('\n',"ROC AUC Score",'\n',roc_auc_score(y, Final_model.predict_proba(X)[:,1]))

[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.6s finished


Performance metrics for Training Data


[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished



 Concordance Matrix 
 col_0          0
is_click        
0         139780
1           9212

 ROC AUC Score 
 0.6385520649750023


[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished


In [35]:
Final_model.feature_importances_

array([0.01519599, 0.06934039, 0.01619393, 0.04282815, 0.03055913,
       0.01095049, 0.03814451, 0.02730694, 0.07944968, 0.10066712,
       0.03311657, 0.03523438, 0.01954062, 0.03484455, 0.02641707,
       0.00148304, 0.00169141, 0.01454955, 0.00505329, 0.00088054,
       0.00800955, 0.0018768 , 0.01018707, 0.00399781, 0.00159248,
       0.00258735, 0.        , 0.00916863, 0.02151686, 0.0032028 ,
       0.00206831, 0.06172731, 0.00437474, 0.00179758, 0.00145322,
       0.00200797, 0.02799263, 0.00154185, 0.        , 0.00405112,
       0.0024913 , 0.00323138, 0.03540801, 0.00510723, 0.00485313,
       0.01514379, 0.02819619, 0.00687278, 0.00772198, 0.        ,
       0.00355266, 0.00034424, 0.00074582, 0.        , 0.00821742,
       0.00545057, 0.00513624, 0.00051674, 0.        , 0.        ,
       0.0001834 , 0.00043574, 0.0021217 , 0.00317304, 0.        ,
       0.00358593, 0.00731498, 0.00722521, 0.01144382, 0.00209607,
       0.        , 0.00104761, 0.00169784, 0.00306249, 0.00669

In [38]:
y=validation_dt["is_click"]
X=validation_dt[['var_1', 'RN', 'Rep_usr', 'Day',
                 'Cum_Click', 
                'user_prod_pref','user_pref',
                 'RN_Camp', 'RN_P',
       'RN_Web', 'RN_day', 'RN_hour', 'RN_Day_hour', 
                 'Day_Cum_Click', 
                 't_diff',
       'product_A', 'product_B', 'product_C', 'product_D', 'product_E',
       'product_F', 'product_G', 'product_H', 'product_I', 'product_J',
       'campaign_id_105960', 'campaign_id_118601', 'campaign_id_359520',
       'campaign_id_360936', 'campaign_id_396664', 'campaign_id_404347',
       'campaign_id_405490', 'campaign_id_414149', 'campaign_id_82320',
       'campaign_id_98970', 'webpage_id_11085', 'webpage_id_13787',
       'webpage_id_1734', 'webpage_id_28529', 'webpage_id_45962',
       'webpage_id_51181', 'webpage_id_53587', 'webpage_id_60305',
       'webpage_id_6970', 'product_category_1_1', 'product_category_1_2',
       'product_category_1_3', 'product_category_1_4', 'product_category_1_5',
       'user_group_id_0.0', 
       #'user_group_id_0.1', 
       'user_group_id_1.0',
       'user_group_id_10.0', 'user_group_id_11.0', 'user_group_id_12.0',
       'user_group_id_2.0', 'user_group_id_3.0', 'user_group_id_4.0',
       'user_group_id_5.0', 'user_group_id_6.0', 'user_group_id_7.0',
       'user_group_id_8.0', 'user_group_id_9.0', 'gender_Female',
       'gender_Male', 
        #'gender_N', 
        'age_level_0.0', 
        #'age_level_0.1',
       'age_level_1.0', 'age_level_2.0', 'age_level_3.0', 'age_level_4.0',
       'age_level_5.0', 'age_level_6.0', 
        #'user_depth_0.0', 
        'user_depth_1.0',
       'user_depth_2.0', 'user_depth_3.0', 
        #'city_development_index_0.0',
       'city_development_index_1.0', 'city_development_index_2.0',
       'city_development_index_3.0', 'city_development_index_4.0', 'hour_0',
       'hour_1', 'hour_10', 'hour_11', 'hour_12', 'hour_13', 'hour_14',
       'hour_15', 'hour_16', 'hour_17', 'hour_18', 'hour_19', 'hour_2',
       'hour_20', 'hour_21', 'hour_22', 'hour_23', 'hour_3', 'hour_4',
       'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9']]

print("Performance metrics for Training Data")
concordance_matrix=pd.crosstab(y, Final_model.predict(X))
#print("Accuracy = ",((concordance_matrix[0][0]+concordance_matrix[1][1])/(concordance_matrix[0][0]+concordance_matrix[1][1]+concordance_matrix[1][0]+concordance_matrix[0][1])))
#print("Precision = ", (concordance_matrix[1][1]/(concordance_matrix[1][1]+concordance_matrix[1][0])))
#print("Recall = ", (concordance_matrix[1][1]/(concordance_matrix[1][1]+concordance_matrix[0][1])))
#print('\n',"Concordance Matrix",'\n',concordance_matrix)
print('\n',"ROC AUC Score",'\n',roc_auc_score(y, Final_model.predict_proba(X)[:,1]))

Performance metrics for Training Data


[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished



 ROC AUC Score 
 0.6385520649750023


[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished


In [39]:
X=testing_dt[['var_1', 'RN', 'Rep_usr', 'Day',
                 'Cum_Click', 
                 'user_prod_pref','user_pref',
                 'RN_Camp', 'RN_P',
       'RN_Web', 'RN_day', 'RN_hour', 'RN_Day_hour', 
                 'Day_Cum_Click', 
                 't_diff',
       'product_A', 'product_B', 'product_C', 'product_D', 'product_E',
       'product_F', 'product_G', 'product_H', 'product_I', 'product_J',
       'campaign_id_105960', 'campaign_id_118601', 'campaign_id_359520',
       'campaign_id_360936', 'campaign_id_396664', 'campaign_id_404347',
       'campaign_id_405490', 'campaign_id_414149', 'campaign_id_82320',
       'campaign_id_98970', 'webpage_id_11085', 'webpage_id_13787',
       'webpage_id_1734', 'webpage_id_28529', 'webpage_id_45962',
       'webpage_id_51181', 'webpage_id_53587', 'webpage_id_60305',
       'webpage_id_6970', 'product_category_1_1', 'product_category_1_2',
       'product_category_1_3', 'product_category_1_4', 'product_category_1_5',
       'user_group_id_0.0', 
              #'user_group_id_0.1', 
              'user_group_id_1.0',
       'user_group_id_10.0', 'user_group_id_11.0', 'user_group_id_12.0',
       'user_group_id_2.0', 'user_group_id_3.0', 'user_group_id_4.0',
       'user_group_id_5.0', 'user_group_id_6.0', 'user_group_id_7.0',
       'user_group_id_8.0', 'user_group_id_9.0', 'gender_Female',
       'gender_Male', 
              #'gender_N', 
              'age_level_0.0', 
              #'age_level_0.1',
       'age_level_1.0', 'age_level_2.0', 'age_level_3.0', 'age_level_4.0',
       'age_level_5.0', 'age_level_6.0', 
              #'user_depth_0.0', 
              'user_depth_1.0',
       'user_depth_2.0', 'user_depth_3.0',
              #'city_development_index_0.0',
       'city_development_index_1.0', 'city_development_index_2.0',
       'city_development_index_3.0', 'city_development_index_4.0', 'hour_0',
       'hour_1', 'hour_10', 'hour_11', 'hour_12', 'hour_13', 'hour_14',
       'hour_15', 'hour_16', 'hour_17', 'hour_18', 'hour_19', 'hour_2',
       'hour_20', 'hour_21', 'hour_22', 'hour_23', 'hour_3', 'hour_4',
       'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9']]

In [40]:
a=pd.DataFrame(Final_model.predict_proba(X)[:,1])
b=testing_dt.join(a, how='outer')
b = b[['session_id',0]]
b.rename(columns={0: 'is_click'}, inplace=True)

[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished


In [41]:
b.to_csv("C:/Users/10644292/AppData/Local/Programs/Python/Python36/Scripts/Data Science Assignments/AMEX/test_LNMuIYp/final_v9.csv", sep=',', index=False)